<a href="https://colab.research.google.com/github/efearase/RL_with_sentiment/blob/main/pulse_parsin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -r requirements.txt

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from httpx import ReadTimeout
from tpulse import TinkoffPulse
from httpx import HTTPStatusError
import time
import os
import requests
from IPython.display import clear_output
from pprint import pp
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

In [3]:
names = pd.read_csv('Export_ru_securities-list_20240129.csv', encoding='windows-1251', delimiter=';', quotechar='"')

In [4]:
codes = names[((names['LIST_SECTION']=='Первый уровень')) & (names['SUPERTYPE']=='Акции')]['TRADE_CODE'].dropna()

In [5]:
date = pd.to_datetime(time.time(), unit='s')
date

Timestamp('2024-03-03 08:52:36.586700032')

In [6]:
def datareader_optimized(ticker):
    cursor = 999999999
    t_0 = time.time()
    KEYS = ["id", "nickname", "content", "inserted"]
    date = pd.to_datetime(time.time(), unit='s').tz_localize('UTC')
    fin_date = pd.to_datetime('2022-03-29').tz_localize('UTC')
    pulse = TinkoffPulse()  # Assuming this is defined elsewhere

    # Initialize an empty DataFrame with specified types if possible
    base_df = pd.DataFrame(columns=["id", "nickname", "text", "inserted"])

    while date >= fin_date:
        try:
            if time.time() - t_0 > 1:
                print(f"{date}, {ticker}")
                clear_output(wait=True)
                t_0 = time.time()
            response = pulse.get_posts_by_ticker(ticker, cursor)
            cursor = response["nextCursor"]
            posts = response["items"]

            chunk_data = []
            for post in posts:
                data = {key: post[key] for key in KEYS if key in post}
                data['text'] = data['content'].get('text', None) if isinstance(data.get('content'), dict) else None
                data['inserted'] = pd.to_datetime(data['inserted']).strftime('%Y-%m-%d')
                del data['content']
                chunk_data.append(data)
                date = pd.to_datetime(post['inserted'])
            if chunk_data:
                chunk_df = pd.DataFrame(chunk_data)
                base_df = pd.concat([base_df, chunk_df], ignore_index=True)

        except (HTTPStatusError, TimeoutError, KeyError, ReadTimeout, NameError, TypeError) as e:
            print(f"Encountered an error: {e}. Continuing...")
            date = pd.to_datetime(date.timestamp() - 24 * 60 * 60, unit = 's').tz_localize('UTC')
            pass

        time.sleep(0.25)

    base_df.to_csv(f"data/text_{ticker}.csv", index=False)
    users = base_df['nickname'].unique()

    return users

In [7]:
def check(name):
    for file in os.listdir('data'):
        if name in file:
            return 1
    return 0

In [11]:
users_set = set()  # Use a set to automatically handle unique entries
for code in codes:
    if not check(code):  # Assuming check is a function defined elsewhere
        new_users = datareader_optimized(code)  # Collect new users from the optimized function
        users_set.update(new_users)  # Efficiently add new users, automatically avoiding duplicates

users = list(users_set)

2022-04-05 13:19:04.017000+03:00, TATNP


In [9]:
pulse = TinkoffPulse()

In [4]:
users_set = set()
for file in os.listdir('text_data/'):
    df = pd.read_csv(f"text_data/{file}")
    new_users = df['nickname'].unique()
    users_set.update(new_users)

In [6]:
users_set

{'IronPiggyBank',
 'ne_fartovyy',
 'Karta_Z',
 'Cofeeld',
 '7emenov',
 'Mehanik77777',
 'investprofiile',
 'Trading_eto_prosto',
 'Ellen_g',
 'Abi_Investor',
 'Arlekino_be_good',
 'Elektrobarbarella',
 'Alllexis',
 'Kupil_USD_po_132',
 'emi_moscow',
 'PKgood',
 'Olaer',
 'Darlik',
 'fabenDumon',
 'scruma',
 'vitally',
 'DeeMAn26',
 'i.zmienko',
 'Evgenio70',
 'Web_Alex',
 'TwoFacedJanus',
 'Maxim2021',
 'BoondockSaint',
 'Miigae4ka',
 'FutureSeeker',
 'g_kirill',
 'Alex_2Net',
 'extratea',
 'V_NeBo',
 'DaPofigMne',
 'MisterChef',
 'Nikol07032201',
 'Dragomip',
 'Winderman',
 'MartinEdwardLee',
 'Renger68',
 'mr_Pavel',
 'NatalyaA22',
 'Nub_sovsem',
 'MikhailSan',
 'Svetlana_love',
 'diablo_omsk',
 'drush7',
 'Jackzzz',
 'Maksimusli',
 'Rainhardl',
 'HUYASOS',
 'kuprup',
 'AlexTrip',
 'MONETNOE.DELO',
 'ForesterHolivar',
 'AirenS',
 'Russyao',
 'degenape',
 'Profit_Powerhouse',
 'YURIVELIK',
 'Sly_Investor',
 'safdon',
 'nastyabutalova',
 'SergeyGolub',
 'st.Lusian',
 'ASHBIASH',
 'k0nd

In [33]:
retries = 5
yields = []
followers = []

for user in users_set:
    print(f"progress: {round(len(followers)/len(users_set)*100, 2)}%")
    clear_output(wait=True)
    try:
        user_info = pulse.get_user_info(user)
        if user_info is not None:
            if 'statistics' in user_info.keys():
                followers.append(user_info['followersCount'])
                yields.append(user_info['statistics']['yearRelativeYield'])
            else:
                followers.append(0)
                yields.append(0)
        else:
                followers.append(0)
                yields.append(0)

    except ReadTimeout or ValueError or TimeoutError or NameError or KeyError:
        followers.append(0)
        yields.append(0)
        time.sleep(5)
            
users_info = pd.DataFrame({'users': users_set, 'yields': yields, 'followers': followers})

users_info.to_csv("users_info.csv")

TypeError: 'set' type is unordered

In [34]:
users_info = pd.DataFrame({'users': list(users_set), 'yields': yields, 'followers': followers})

users_info.to_csv("users_info.csv")

In [18]:
followers

[14,
 14,
 14,
 14,
 14,
 4,
 4,
 4,
 4,
 4,
 1,
 1,
 1,
 1,
 1,
 95,
 95,
 95,
 95,
 95,
 16,
 16,
 16,
 16,
 16,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 70,
 70,
 70,
 70,
 70,
 9,
 9,
 9,
 9,
 9,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 14,
 14,
 14,
 14,
 14,
 4,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 47,
 47,
 47,
 47,
 47,
 0,
 0,
 0,
 0,
 0,
 443,
 443,
 443,
 443,
 443,
 178,
 178,
 178,
 178,
 178,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 14,
 14,
 14,
 14,
 14,
 84,
 84,
 84,
 84,
 84,
 10,
 10,
 10,
 10,
 10,
 0,
 0,
 0,
 0,
 0,
 6,
 6,
 6,
 6,
 6,
 0,
 0,
 0,
 0,
 3,
 3,
 3,
 3,
 3,
 8,
 8,
 8,
 8,
 8,
 0,
 0,
 0,
 0,
 0,
 250,
 250,
 250,
 250,
 250,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 13,
 13,
 13,
 13,
 1,
 1,
 1,
 1,
 12,
 12,
 12,
 12,
 12,
 4,
 4,
 4,
 4,
 4,
 19,
 19,
 19,
 19,
 19,
 39,
 39,
 39,
 39,
 39,
 70,
 70,
 70,
 70,
 70,
 3,
 3,
 3,
 3,
 41,
 41,
 41,
 41,
 41,
 13,
 13,
 13,
 13,
 13,
 3,
 3,
 3,
 3,
 3,
 116,
 116,
 116,
 116,
 11

In [11]:
users_info.to_csv("users_info.csv")

NameError: name 'users_info' is not defined